In [ ]:
def cleanTweet(tw_list):

  #Renombramos las columnas
  n = tw_list.columns[1]
  tw_list.columns = ['original',n]

  #Quitamos las filas con NA en clase
  tw_list = tw_list[tw_list[n].notna()]

  #Quitar RT, puntuación, links, etc
  remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
  rt = lambda x: re.sub("(@[A-Za-z0–9]+)|([^0-9A-Za-zÀ-ÿ \t])|(\w+:\/\/\S+)"," ",x)
  tw_list.original = tw_list.original.map(remove_rt).map(rt)

  #Convertir a minúsculas
  tw_list.original = tw_list.original.str.lower()

  #Quitar duplicado y resetear indices
  tw_list.drop_duplicates(inplace=True)
  tw_list = tw_list.reset_index(drop=True)

  #No se en que momento se pierde el DF, seguro en el map
  tw_list = pd.DataFrame(tw_list)
  return tw_list

In [ ]:
def sentimentAnalysis(tw_list):
  #tw_list.columns = ['original']

  translated_list = []
  for tweet in tw_list['original']:

    api_url = "http://mymemory.translated.net/api/get?de=gabrielfjrfi@gmail.com&q={}&langpair={}|{}".format(tweet, 'ca','en')
    hdrs = {
                    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
                    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                    'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                    'Accept-Encoding': 'none',
                    'Accept-Language': 'en-US,en;q=0.8',
                    'Connection': 'keep-alive'
            }
    response = requests.get(api_url, headers=hdrs)
    response_json = json.loads(response.text)
    translation = response_json["responseData"]["translatedText"]
    translator_name = "MemoryNet Translation Service"
    translated_list.append(translation)

  #Crea una segunda columna con la traducción, entonces la primera tiene el original y la segunda el traducido
  tw_list['traduccion'] = pd.DataFrame(translated_list)

  tw_list[['polarity', 'subjectivity']] = tw_list['traduccion'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
  for index, row in tw_list['traduccion'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
      tw_list.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
      tw_list.loc[index, 'sentiment'] = "positive"
    else:
      tw_list.loc[index, 'sentiment'] = "neutral"

    tw_list.loc[index, 'neg'] = neg
    tw_list.loc[index, 'neu'] = neu
    tw_list.loc[index, 'pos'] = pos
    tw_list.loc[index, 'compound'] = comp

  return tw_list